# YFinanceCrawler

1. Crawl monthly data (default is 10 years back) on Yahoo Finance
2. Store and read from persistent shelve
3. Use FinQuant to construct and analyze portfolio

In [232]:
import shelve
import re
import string
import time
import os
from datetime import date, datetime, timedelta
from io import StringIO

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import finquant
from finquant.portfolio import build_portfolio
from portfolios import *

%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [228]:
class YFinanceCrawler:
    timeout = 2
    crumb_link = "https://finance.yahoo.com/quote/{0}/history?p={0}"
    crumble_regex = r'CrumbStore":{"crumb":"(.*?)"}'
    quote_link = "https://query1.finance.yahoo.com/v7/finance/download/{quote}?period1={dfrom}&period2={dto}&interval=1mo&events=history&crumb={crumb}"
    fpath = os.path.join("data", "master.p")

    def __init__(self, tickers, years_back=10):
        self.tickers = tickers
        self.session = requests.Session()
        self.dateto = date.today()
        self.datefrom = date(
            self.dateto.year - years_back, self.dateto.month, self.dateto.day
        )

    def returnDateIndex(self):
        print(self.dateIndex)

    def get_crumb(self, ticker):
        response = self.session.get(
            self.crumb_link.format(ticker), timeout=self.timeout
        )
        response.raise_for_status()
        match = re.search(self.crumble_regex, response.text)
        if not match:
            raise ValueError("Could not get crumb from Yahoo Finance")
        else:
            self.crumb = match.group(1)

    def get_quote(self, ticker):
        if not hasattr(self, "crumb") or len(self.session.cookies) == 0:
            self.get_crumb(ticker)
        url = self.quote_link.format(
            quote=ticker,
            dto=int(time.mktime(self.dateto.timetuple())),
            dfrom=int(time.mktime(self.datefrom.timetuple())),
            crumb=self.crumb,
        )
        response = self.session.get(url)
        response.raise_for_status()

        data = pd.read_csv(StringIO(response.text), parse_dates=["Date"])
        dateIndex = data["Date"]
        return (data, dateIndex)

    def getPeriodIndex(self):

        return pd.date_range(start=self.datefrom, end=self.dateto, freq="M")

    def getDummyDateIndex(self, periods=120):
        dateto = date.today()
        datefrom = datetime(2020, 5, 5)

        return pd.date_range(start=datefrom, end=dateto, periods=periods)

    def getPricesData(self, update=False):

        try:
            master = shelve.open(self.fpath)
        except ValueError:
            print("cannot open master shelve")

        if update:
            print("forced update of master")
            return self.queryNewData(self.tickers, master)

        # if file is not empty & is up-to-date
        if "ver" in master and master["ver"] == self.dateto:
            print("reading today's version \n")

            read_master = dict()
            read = []
            newTickers = []
            index = []

            # read from master
            for ticker in self.tickers:

                # if master contains ticker
                if ticker in master["data"]:
                    read_master[ticker] = master["data"][ticker]
                    read.append(ticker)
                    index = master["data"][ticker]["Date"]

                # if master does not contain ticker
                else:
                    newTickers.append(ticker)

            if newTickers:
                print("querying new tickers \n")
                query_master, index = self.queryNewData(newTickers, master)
                new_master = {**read_master, **query_master}
            else:
                new_master = read_master

            print("read: ", read)
            master.close()
            return new_master, index

        # if file is empty
        else:
            print("creating new master, either due to new file or outdated file \n")
            return self.queryNewData(self.tickers, master)

    def queryNewData(self, tickers, master):

        new_master = dict()
        query = []

        for ticker in tickers:
            data, index = self.get_quote(ticker)
            #             data = list(np.random.randint(100, size=120))
            query.append(ticker)

            new_master[ticker] = data

        master["ver"] = date.today()
        master["data"] = new_master
        print("queried: ", query)
        master.close()
        return new_master, index

    def buildDf(self, update=False):
        data, index = self.getPricesData(update)
        adjClose = dict()
        for d in iter(data):
            adjClose[d] = data[d]["Adj Close"]

        df = pd.DataFrame(data=adjClose).set_index(index)

        return df


def buildPf(df):
    pf = build_portfolio(data=df)
    return pf


def stats(pf):
    vol = pf.comp_volatility()
    sha = pf.comp_sharpe()
    ret = pf.comp_expected_return(freq=12)

    print("volatility = {:f}\b".format(vol))
    print("sharpe = {:f}\b".format(sha))
    print("return = {:f}\b\n".format(ret))


def castFloat(df, col):
    punct = string.punctuation.replace("|", "")  # to use `|` as separator
    transtab = str.maketrans(dict.fromkeys(punct, ""))

    for c in col:
        df[c] = "|".join(df[c].tolist()).translate(transtab).split("|")
    df = df.astype(dict.fromkeys(col, "f"))
    return df

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [229]:
yfc = YFinanceCrawler(pf[0])
# data = yfc1.getPricesData()
df = yfc.buildDf()

reading today's version 

querying new tickers 

queried:  ['VTSAX', 'XAR', 'VTV', 'VGT', 'XLF', 'AAPL', 'UNH', 'MA', 'V', 'INTC', 'NKE', 'MSFT']
read:  ['BND']


In [187]:
fpath = os.path.join("data", "master.p")
master = shelve.open(fpath)
print(master["data"])
master.close()

{'BND':           Date       Open       High        Low      Close  Adj Close  \
0   2010-08-01  81.610001  82.980003  81.510002  82.889999  62.308945   
1   2010-09-01  82.580002  82.830002  81.720001  82.660004  62.312706   
2   2010-10-01  82.389999  83.059998  82.220001  82.699997  62.513298   
3   2010-11-01  82.639999  83.000000  81.260002  81.910004  62.083572   
4   2010-12-01  81.360001  81.519997  79.519997  80.269997  61.008766   
..         ...        ...        ...        ...        ...        ...   
115 2020-03-01  86.820000  89.320000  76.489998  85.349998  84.502914   
116 2020-04-01  86.120003  87.860001  85.550003  87.510002  86.814590   
117 2020-05-01  87.379997  87.940002  86.629997  87.919998  87.415924   
118 2020-06-01  87.589996  88.360001  87.029999  88.339996  88.008499   
119 2020-07-01  88.139999  88.199997  88.000000  88.199997  88.038254   

        Volume  
0     19117600  
1     12817000  
2     15738800  
3     16326800  
4     18661200  
..         ..

In [230]:
df

,BND,VTSAX,XAR,VTV,VGT,XLF,AAPL,UNH,MA,V,INTC,NKE,MSFT
Date,,,,,,,,,,,,,
2010-08-01,62.308945,21.488342,21.868479,35.117844,43.913059,7.057582,30.064436,27.075375,17.074158,14.338008,13.028164,13.215457,18.540930
2010-09-01,62.312706,23.407389,24.508539,37.521702,49.474487,7.468754,35.091660,29.968992,19.281160,15.538717,14.264233,15.129810,19.449926
2010-10-01,62.513298,24.455280,24.209326,38.808945,52.642284,7.595080,37.222515,30.884825,20.663555,16.354773,14.895717,15.608957,21.181273
2010-11-01,62.083572,24.596022,24.636147,38.568470,52.143990,7.542918,38.480263,31.287479,20.540718,15.452919,15.720369,16.507853,20.061455
2010-12-01,61.008766,26.135855,25.851347,41.368809,54.742287,8.320162,39.891342,30.936232,19.421095,14.845623,15.746099,16.371773,22.302277
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-01,84.502914,62.163616,NaN,87.578484,210.558502,20.514080,253.603500,247.239609,241.187042,160.850662,53.816006,82.534111,157.271988
2020-04-01,86.814590,70.758240,NaN,97.787567,241.400726,22.645544,293.006836,291.215759,274.545471,178.421249,59.643089,86.963066,178.712280
2020-05-01,87.415924,74.561485,NaN,100.605309,260.383606,23.261614,317.081665,303.542664,300.890015,194.913635,62.576523,98.334702,182.741058


In [231]:
pf = buildPf(df)
stats(pf)

volatility = 0.52574
sharpe = 7.99663
return = 0.20043

